IMPORT

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.decomposition import TruncatedSVD,NMF,FactorAnalysis,KernelPCA,FastICA
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import os 
import gc
import random
sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)
set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None

In [2]:
class CFG:
    folds=5
    seed=1
    train=False
    ver='git_'
    kfold='sk'
    model='lgb'
    n_components=10
    drop_col=[]
    cluster_num=4
    num_iterations=100000
def fix_seed(seed=CFG.seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
fix_seed()

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv('../data/test.csv')

In [4]:
def describe_satatistics(input_df):
    desc = pd.DataFrame(index=list(input_df))
    desc["nunique"] = train.nunique()
    desc["unique%"] = train.nunique() / len(input_df)
    desc["null"] = train.isnull().sum()
    desc["type"] = train.dtypes
    desc = pd.concat([desc,input_df.describe().T],axis=1)
    return desc

# Preprocessing

In [5]:
train_df = train.copy()
test_df = test.copy()

In [6]:
train["train_data"]=1
test["train_data"]=0
eda_df = pd.concat([train,test])
eda_df.reset_index(inplace=True,drop=True)

In [7]:
train_df_0 = train_df[train_df["UrbanRural"]==0]
train_df_0.reset_index(drop=True,inplace=True)
train_df_12 = train_df[(train_df["UrbanRural"]==1) |(train_df["UrbanRural"]==2) ]
train_df_12.reset_index(drop=True,inplace=True)
test_df_0 = test_df[test_df["UrbanRural"]==0]
test_df_0.reset_index(drop=True,inplace=True)
test_df_12 = test_df[(test_df["UrbanRural"]==1) |(test_df["UrbanRural"]==2) ]
test_df_12.reset_index(drop=True,inplace=True)

In [8]:
dataframe_list = [eda_df,train_df,test_df,test_df_0,test_df_12,train_df_0,train_df_12]

In [9]:
pd.set_option('display.max_columns', None)
eda_df.head()

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,train_data
0,0,163,21,1.0,0,0,1,N,N,31-Jan-98,1.0,0,22-Sep-06,2006,PHOENIX,AZ,SD,"$80,000.00","$80,000.00","$68,000.00",0,1
1,1,84,6,1.0,4,0,0,0,N,31-Oct-93,1.0,62,30-Jun-92,1992,MCALESTER,OK,OK,"$287,000.00","$287,000.00","$229,600.00",0,1
2,2,242,45,1.0,4,90,0,N,N,31-Aug-01,1.0,42,18-Apr-01,2001,HAWTHORNE,NJ,NJ,"$31,983.00","$30,000.00","$15,000.00",1,1
3,3,237,4,1.0,0,0,0,N,N,31-Aug-07,1.0,33,6-Oct-03,2004,NASHVILLE,TN,SD,"$229,000.00","$229,000.00","$229,000.00",0,1
4,4,184,0,1.0,0,0,0,N,N,8-Jun-83,1.0,0,17-Dec-99,2000,POMONA,CA,CA,"$525,000.00","$525,000.00","$393,750.00",0,1


In [10]:
def clean_date(input_df,col):
    input_df[col] = pd.to_datetime(input_df[col])
    return input_df

In [11]:
for c in dataframe_list:
    for a in ["DisbursementDate","ApprovalDate"]:
        c = clean_date(c,a)

C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_11608\2940027403.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  input_df[col] = pd.to_datetime(input_df[col])
C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_11608\2940027403.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  input_df[col] = pd.to_datetime(input_df[col])


C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_11608\2940027403.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  input_df[col] = pd.to_datetime(input_df[col])
C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_11608\2940027403.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  input_df[col] = pd.to_datetime(input_df[col])
C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_11608\2940027403.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  input_df[col] = pd.to_datetime(input_df[col])
C:\Users\kaminogo.kenji\AppData\Local\Temp\ipykernel_11608\2940027403.py:2: 

In [12]:
le = LabelEncoder()
label_columns_list = ["RevLineCr","LowDoc","City","State","BankState"]
def label_encode(input_df):
    for col in label_columns_list:
        le = LabelEncoder()
        le.fit(input_df[col])
        input_df[f"{col}_label"] = le.transform(input_df[col])
    return input_df


In [13]:
def clean_money(input_df):
    money_columns = ["DisbursementGross","GrAppv","SBA_Appv"]
    for col in money_columns:
        input_df[col] = input_df[col].str.replace('$', '').replace(',', '')
        input_df[col] = input_df[col].str.replace(",","")
        input_df[col] = input_df[col].str.extract(r"(\d+)")
        input_df[col] = input_df[col].astype(int)
    return input_df

In [14]:
def create_NoEmpLabel(input_df):
    input_df["NoEmp_label"] = 0
    input_df["NoEmp_label"] = np.where(input_df["NoEmp"]<3,1,input_df["NoEmp_label"])
    input_df["NoEmp_label"] = np.where((input_df["NoEmp"]>=3) & (input_df["NoEmp"]<5),2,input_df["NoEmp_label"])
    input_df["NoEmp_label"] = np.where((input_df["NoEmp"]>=5 )& (input_df["NoEmp"]<13),3,input_df["NoEmp_label"])
    input_df["NoEmp_label"] = np.where(input_df["NoEmp"]>=13,4,input_df["NoEmp_label"])
    print(input_df["NoEmp_label"].value_counts())
    return input_df

In [15]:
test_df_0.dtypes

Unnamed: 0                    int64
Term                          int64
NoEmp                         int64
NewExist                    float64
CreateJob                     int64
RetainedJob                   int64
FranchiseCode                 int64
RevLineCr                    object
LowDoc                       object
DisbursementDate     datetime64[ns]
Sector                        int64
ApprovalDate         datetime64[ns]
ApprovalFY                    int64
City                         object
State                        object
BankState                    object
DisbursementGross            object
GrAppv                       object
SBA_Appv                     object
UrbanRural                    int64
dtype: object

In [16]:
for c in dataframe_list:
        c = clean_money(c)
        c = label_encode(c)
        c = create_NoEmpLabel(c)

NoEmp_label
1    25873
2    20864
4    20698
3    17180
Name: count, dtype: int64
NoEmp_label
1    12945
2    10465
4    10332
3     8565
Name: count, dtype: int64
NoEmp_label
1    12928
2    10399
4    10366
3     8615
Name: count, dtype: int64
NoEmp_label
4    7400
1    5841
3    5817
2    5286
Name: count, dtype: int64
NoEmp_label
1    7087
2    5113
4    2966
3    2798
Name: count, dtype: int64
NoEmp_label
4    7301
1    5735
3    5733
2    5268
Name: count, dtype: int64
NoEmp_label
1    7210
2    5197
4    3031
3    2832
Name: count, dtype: int64


In [17]:
def create_main_key(input_df,column):
    tmp_df = input_df.copy()
    tmp_df[f"main_key_{column}"] = tmp_df[f"{column}"].astype(str).str.cat(tmp_df["Sector"].astype(str), sep="_")
    count_df =pd.DataFrame(tmp_df[f"main_key_{column}"].value_counts())
    count_df.reset_index(inplace = True)
    top_key_list = count_df[f"main_key_{column}"].head(100)
    input_df[f"main_key_{column}"] = input_df[f"{column}"].astype(str).str.cat(input_df["Sector"].astype(str), sep="_")
    input_df[f"main_key_{column}"] = np.where(input_df[f"main_key_{column}"].isin(top_key_list),input_df[f"main_key_{column}"],"XX")
    input_df[f"main_key_{column}"].value_counts()
    return input_df

In [18]:
main_key_list = []
for i,dataFrame in enumerate(dataframe_list):
    for col in ["NoEmp_label","State","City"]:
            dataFrame= create_main_key(dataFrame,col)
            if i == 0:
                main_key_list.append(f"main_key_{col}")
            print(col,"completed")

NoEmp_label completed
State completed
City completed
NoEmp_label completed
State completed
City completed
NoEmp_label completed
State completed
City completed
NoEmp_label completed
State completed
City completed
NoEmp_label completed
State completed
City completed
NoEmp_label completed
State completed
City completed
NoEmp_label completed
State completed
City completed


In [19]:
main_key_list

['main_key_NoEmp_label', 'main_key_State', 'main_key_City']

In [20]:
def create_groupby_features(input_df,target_col,group_feature):
    tmp_df =  input_df.groupby(group_feature)[target_col].agg(["median"])
    tmp_df.reset_index(inplace=True)
    tmp_df.rename(columns = {"median":f"{target_col}_{group_feature}_med"},inplace=True)
    if group_feature=='main_key':
        tmp_df[group_feature]=np.where(tmp_df[group_feature]=='XX','ZZZ',tmp_df[group_feature])
    input_df = pd.merge(input_df,tmp_df,how="left",on=group_feature)
    input_df[f"{target_col}_{group_feature}_diff"] = input_df[target_col] - input_df[f"{target_col}_{group_feature}_med"]
    print("completed")
    return input_df[[f"{target_col}_{group_feature}_med",f"{target_col}_{group_feature}_diff"]]

In [21]:
for c in dataframe_list:
    for key in main_key_list:
        for col in ["DisbursementGross","GrAppv","SBA_Appv","CreateJob","RetainedJob","NoEmp"]:
            c[[f"{col}_{key}_med",f"{col}_{key}_diff"]] = create_groupby_features(c,col,key)


completed


completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed

In [22]:
eda_df

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,train_data,RevLineCr_label,LowDoc_label,City_label,State_label,BankState_label,NoEmp_label,main_key_NoEmp_label,main_key_State,main_key_City,DisbursementGross_main_key_NoEmp_label_med,DisbursementGross_main_key_NoEmp_label_diff,GrAppv_main_key_NoEmp_label_med,GrAppv_main_key_NoEmp_label_diff,SBA_Appv_main_key_NoEmp_label_med,SBA_Appv_main_key_NoEmp_label_diff,CreateJob_main_key_NoEmp_label_med,CreateJob_main_key_NoEmp_label_diff,RetainedJob_main_key_NoEmp_label_med,RetainedJob_main_key_NoEmp_label_diff,NoEmp_main_key_NoEmp_label_med,NoEmp_main_key_NoEmp_label_diff,DisbursementGross_main_key_State_med,DisbursementGross_main_key_State_diff,GrAppv_main_key_State_med,GrAppv_main_key_State_diff,SBA_Appv_main_key_State_med,SBA_Appv_main_key_State_diff,CreateJob_main_key_State_med,CreateJob_main_key_State_diff,RetainedJob_main_key_State_med,RetainedJob_main_key_State_diff,NoEmp_main_key_State_med,NoEmp_main_key_State_diff,DisbursementGross_main_key_City_med,DisbursementGross_main_key_City_diff,GrAppv_main_key_City_med,GrAppv_main_key_City_diff,SBA_Appv_main_key_City_med,SBA_Appv_main_key_City_diff,CreateJob_main_key_City_med,CreateJob_main_key_City_diff,RetainedJob_main_key_City_med,RetainedJob_main_key_City_diff,NoEmp_main_key_City_med,NoEmp_main_key_City_diff
0,0,163,21,1.0,0,0,1,N,N,1998-01-31,1.0,0,2006-09-22,2006,PHOENIX,AZ,SD,80000,80000,68000,0,1,1,3,2208,3,42,4,4_0,AZ_0,PHOENIX_0,100000.0,-20000.0,100000.0,-20000.0,80000.0,-12000.0,0.0,0.0,0.0,0.0,19.0,2.0,80000.0,0.0,80000.0,0.0,68000.0,0.0,0.0,0.0,0.0,0.0,6.0,15.0,80000.0,0.0,80000.0,0.0,68000.0,0.0,0.0,0.0,0.0,0.0,10.0,11.0
1,1,84,6,1.0,4,0,0,0,N,1993-10-31,1.0,62,1992-06-30,1992,MCALESTER,OK,OK,287000,287000,229600,0,1,0,3,1723,36,36,3,3_62,XX,XX,120000.0,167000.0,100000.0,187000.0,76500.0,153100.0,0.0,4.0,0.0,0.0,6.0,0.0,99000.0,188000.0,83000.0,204000.0,57600.0,172000.0,0.0,4.0,0.0,0.0,4.0,2.0,99000.0,188000.0,84000.0,203000.0,58500.0,171100.0,0.0,4.0,0.0,0.0,4.0,2.0
2,2,242,45,1.0,4,90,0,N,N,2001-08-31,1.0,42,2001-04-18,2001,HAWTHORNE,NJ,NJ,31983,30000,15000,1,1,1,3,1214,31,31,4,4_42,NJ_42,XX,116664.0,-84681.0,91000.0,-61000.0,58100.0,-43100.0,0.0,4.0,0.0,90.0,18.0,27.0,115000.0,-83017.0,100000.0,-70000.0,60350.0,-45350.0,0.0,4.0,0.0,90.0,13.0,32.0,99000.0,-67017.0,84000.0,-54000.0,58500.0,-43500.0,0.0,4.0,0.0,90.0,4.0,41.0
3,3,237,4,1.0,0,0,0,N,N,2007-08-31,1.0,33,2003-10-06,2004,NASHVILLE,TN,SD,229000,229000,229000,0,1,1,3,1906,42,42,2,2_33,TN_33,NASHVILLE_33,98000.0,131000.0,83000.0,146000.0,55800.0,173200.0,0.0,0.0,1.0,-1.0,4.0,0.0,121000.0,108000.0,121000.0,108000.0,88500.0,140500.0,0.0,0.0,1.0,-1.0,6.0,-2.0,154202.5,74797.5,139000.0,90000.0,115500.0,113500.0,0.0,0.0,0.0,0.0,6.0,-2.0
4,4,184,0,1.0,0,0,0,N,N,1983-06-08,1.0,0,1999-12-17,2000,POMONA,CA,CA,525000,525000,393750,0,1,1,3,2246,4,4,1,1_0,CA_0,POMONA_0,80000.0,445000.0,80000.0,445000.0,68000.0,325750.0,0.0,0.0,0.0,0.0,1.0,-1.0,110000.0,415000.0,110000.0,415000.0,90000.0,303750.0,0.0,0.0,0.0,0.0,4.0,-4.0,243000.0,282000.0,243000.0,282000.0,183750.0,210000.0,0.0,0.0,0.0,0.0,2.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84610,84610,243,10,1.0,3,14,0,N,N,2012-12-01,NaN,42,2012-04-23,2012,FT. WRIGHT,KY,OH,390000,150000,127500,0,0,1,3,1022,17,35,3,3_42,XX,XX,104840.0,285160.0,92000.0,58000.0,62000.0,65500.0,0.0,3.0,0.0,14.0,6.0,4.0,99000.0,291000.0,83000.0,67000.0,57600.0,69900.0,0.0,3.0,0.0,14.0,4.0,6.0,99000.0,291000.0,84000.0,66000.0,58500.0,69000.0,0.0,3.0,0.0,14.0,4.0,6.0
84611,84611,178,0,2.0,0,0,1,N,N,2003-11-30,NaN,0,2006-10-27,2007,PHILADELPHIA,PA,RI,100000,100000,900

In [23]:
train_df

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,RevLineCr_label,LowDoc_label,City_label,State_label,BankState_label,NoEmp_label,main_key_NoEmp_label,main_key_State,main_key_City,DisbursementGross_main_key_NoEmp_label_med,DisbursementGross_main_key_NoEmp_label_diff,GrAppv_main_key_NoEmp_label_med,GrAppv_main_key_NoEmp_label_diff,SBA_Appv_main_key_NoEmp_label_med,SBA_Appv_main_key_NoEmp_label_diff,CreateJob_main_key_NoEmp_label_med,CreateJob_main_key_NoEmp_label_diff,RetainedJob_main_key_NoEmp_label_med,RetainedJob_main_key_NoEmp_label_diff,NoEmp_main_key_NoEmp_label_med,NoEmp_main_key_NoEmp_label_diff,DisbursementGross_main_key_State_med,DisbursementGross_main_key_State_diff,GrAppv_main_key_State_med,GrAppv_main_key_State_diff,SBA_Appv_main_key_State_med,SBA_Appv_main_key_State_diff,CreateJob_main_key_State_med,CreateJob_main_key_State_diff,RetainedJob_main_key_State_med,RetainedJob_main_key_State_diff,NoEmp_main_key_State_med,NoEmp_main_key_State_diff,DisbursementGross_main_key_City_med,DisbursementGross_main_key_City_diff,GrAppv_main_key_City_med,GrAppv_main_key_City_diff,SBA_Appv_main_key_City_med,SBA_Appv_main_key_City_diff,CreateJob_main_key_City_med,CreateJob_main_key_City_diff,RetainedJob_main_key_City_med,RetainedJob_main_key_City_diff,NoEmp_main_key_City_med,NoEmp_main_key_City_diff
0,0,163,21,1.0,0,0,1,N,N,1998-01-31,1,0,2006-09-22,2006,PHOENIX,AZ,SD,80000,80000,68000,0,1,3,1857,3,41,4,4_0,AZ_0,PHOENIX_0,100000.0,-20000.0,100000.0,-20000.0,82500.0,-14500.0,0.0,0.0,0.0,0.0,19.0,2.0,80000.0,0.0,80000.0,0.0,68000.0,0.0,0.0,0.0,0.0,0.0,7.0,14.0,80000.0,0.0,80000.0,0.0,68000.0,0.0,0.0,0.0,0.0,0.0,10.0,11.0
1,1,84,6,1.0,4,0,0,0,N,1993-10-31,1,62,1992-06-30,1992,MCALESTER,OK,OK,287000,287000,229600,0,0,3,1451,36,36,3,3_62,XX,XX,121000.0,166000.0,100000.0,187000.0,80000.0,149600.0,0.0,4.0,0.0,0.0,6.0,0.0,99000.0,188000.0,82500.0,204500.0,56270.0,173330.0,0.0,4.0,0.0,0.0,4.0,2.0,100000.0,187000.0,85000.0,202000.0,59500.0,170100.0,0.0,4.0,0.0,0.0,4.0,2.0
2,2,242,45,1.0,4,90,0,N,N,2001-08-31,1,42,2001-04-18,2001,HAWTHORNE,NJ,NJ,31983,30000,15000,1,1,3,1021,31,31,4,4_42,NJ_42,XX,121050.0,-89067.0,100000.0,-70000.0,65600.0,-50600.0,0.0,4.0,0.0,90.0,18.0,27.0,110000.0,-78017.0,100000.0,-70000.0,76500.0,-61500.0,0.0,4.0,0.0,90.0,12.0,33.0,100000.0,-68017.0,85000.0,-55000.0,59500.0,-44500.0,0.0,4.0,0.0,90.0,4.0,41.0
3,3,237,4,1.0,0,0,0,N,N,2007-08-31,1,33,2003-10-06,2004,NASHVILLE,TN,SD,229000,229000,229000,0,1,3,1610,42,41,2,2_33,TN_33,NASHVILLE_33,98000.0,131000.0,82050.0,146950.0,56000.0,173000.0,0.0,0.0,1.0,-1.0,4.0,0.0,118000.0,111000.0,111600.0,117400.0,77925.0,151075.0,0.0,0.0,1.0,-1.0,5.5,-1.5,158300.0,70700.0,168300.0,60700.0,132000.0,97000.0,0.0,0.0,0.0,0.0,6.0,-2.0
4,4,184,0,1.0,0,0,0,N,N,1983-06-08,1,0,1999-12-17,2000,POMONA,CA,CA,525000,525000,393750,0,1,3,1893,4,4,1,1_0,CA_0,POMONA_0,80000.0,445000.0,80000.0,445000.0,68000.0,325750.0,0.0,0.0,0.0,0.0,1.0,-1.0,110000.0,415000.0,110000.0,415000.0,90000.0,303750.0,0.0,0.0,0.0,0.0,4.0,-4.0,275000.0,250000.0,275000.0,250000.0,243000.0,150750.0,0.0,0.0,0.0,0.0,3.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42302,42302,283,14,1.0,0,0,1,N,N,1998-01-31,1,0,1995-03-02,1995,PHILADELPHIA,PA,PA,80000,80000,68000,0,1,3,1856,38,38,4,4_0,PA_0,PHILADELPHIA_0,100000.0,-20000.0,100000.0,-20000.0,82500.0,-14500.0,0.0,0.0,0.0,0.0,19.0,-5.0,80000.0,0.0,80000.0,0.0,64000.0,4000.0,0.0,0.0,0.0,0.0,4.0,10.0,80000.0,0.0,80000.0,0.0,68000.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0
42303,42303,53,2,1.0,0,0,0,Y,N,1991-04-03,1,42,2007-06-06,2007,LOS ANGELES,CA,SD,5000,5000,4250,1,3,3,1349,4,41,1,1_42,CA_42,LOS ANGELES_42,61937.0,

In [24]:
test_df

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,RevLineCr_label,LowDoc_label,City_label,State_label,BankState_label,NoEmp_label,main_key_NoEmp_label,main_key_State,main_key_City,DisbursementGross_main_key_NoEmp_label_med,DisbursementGross_main_key_NoEmp_label_diff,GrAppv_main_key_NoEmp_label_med,GrAppv_main_key_NoEmp_label_diff,SBA_Appv_main_key_NoEmp_label_med,SBA_Appv_main_key_NoEmp_label_diff,CreateJob_main_key_NoEmp_label_med,CreateJob_main_key_NoEmp_label_diff,RetainedJob_main_key_NoEmp_label_med,RetainedJob_main_key_NoEmp_label_diff,NoEmp_main_key_NoEmp_label_med,NoEmp_main_key_NoEmp_label_diff,DisbursementGross_main_key_State_med,DisbursementGross_main_key_State_diff,GrAppv_main_key_State_med,GrAppv_main_key_State_diff,SBA_Appv_main_key_State_med,SBA_Appv_main_key_State_diff,CreateJob_main_key_State_med,CreateJob_main_key_State_diff,RetainedJob_main_key_State_med,RetainedJob_main_key_State_diff,NoEmp_main_key_State_med,NoEmp_main_key_State_diff,DisbursementGross_main_key_City_med,DisbursementGross_main_key_City_diff,GrAppv_main_key_City_med,GrAppv_main_key_City_diff,SBA_Appv_main_key_City_med,SBA_Appv_main_key_City_diff,CreateJob_main_key_City_med,CreateJob_main_key_City_diff,RetainedJob_main_key_City_med,RetainedJob_main_key_City_diff,NoEmp_main_key_City_med,NoEmp_main_key_City_diff
0,42307,5,2,1.0,1,0,0,T,N,2004-07-31,23,2007-08-06,2007,SUNNYVALE,CA,CA,25000,25000,21250,1,2,3,2308,4,4,1,1_23,CA_23,XX,65000.0,-40000.0,60000.0,-35000.0,40705.0,-19455.0,0.0,1.0,1.0,-1.0,1.0,1.0,70000.0,-45000.0,62200.0,-37200.0,43375.0,-22125.0,0.0,1.0,0.0,0.0,3.0,-1.0,98500.0,-73500.0,83000.0,-58000.0,57600.0,-36350.0,0.0,1.0,0.0,0.0,4.0,-2.0
1,42308,235,13,1.0,9,14,77725,Y,N,1995-01-11,44,2004-03-08,2004,PITTSBURGH,PA,PA,15000,15000,7500,0,3,3,1862,38,38,4,4_44,PA_44,XX,130150.0,-115150.0,108100.0,-93100.0,84480.0,-76980.0,0.0,9.0,0.0,14.0,18.0,-5.0,120600.0,-105600.0,100900.0,-85900.0,76000.0,-68500.0,0.0,9.0,0.0,14.0,4.0,9.0,98500.0,-83500.0,83000.0,-68000.0,57600.0,-50100.0,0.0,9.0,0.0,14.0,4.0,9.0
2,42309,31,5,2.0,0,0,0,N,NaN,NaT,56,2007-02-27,2007,LITTLE ROCK,AR,AR,28000,28000,23800,1,1,6,1311,2,2,3,3_56,XX,XX,119300.0,-91300.0,100000.0,-72000.0,83678.0,-59878.0,0.0,0.0,0.0,0.0,6.0,-1.0,96200.0,-68200.0,80000.0,-52000.0,56250.0,-32450.0,0.0,0.0,0.0,0.0,4.0,1.0,98500.0,-70500.0,83000.0,-55000.0,57600.0,-33800.0,0.0,0.0,0.0,0.0,4.0,1.0
3,42310,120,4,1.0,0,1,0,Y,N,1999-04-30,62,1997-12-19,1998,LITTLE ROCK,AR,AR,7500,7500,6375,2,3,3,1311,2,2,2,2_62,XX,XX,100000.0,-92500.0,98500.0,-91000.0,64000.0,-57625.0,0.0,0.0,0.0,1.0,3.0,1.0,96200.0,-88700.0,80000.0,-72500.0,56250.0,-49875.0,0.0,0.0,0.0,1.0,4.0,0.0,98500.0,-91000.0,83000.0,-75500.0,57600.0,-51225.0,0.0,0.0,0.0,1.0,4.0,0.0
4,42311,63,13,1.0,0,8,1,N,N,2005-12-31,42,2009-07-10,2009,Louisville,KY,DE,91000,93000,93000,0,1,3,1374,17,8,4,4_42,XX,XX,111600.0,-20600.0,84500.0,8500.0,54900.0,38100.0,0.0,0.0,0.0,8.0,18.0,-5.0,96200.0,-5200.0,80000.0,13000.0,56250.0,36750.0,0.0,0.0,0.0,8.0,4.0,9.0,98500.0,-7500.0,83000.0,10000.0,57600.0,35400.0,0.0,0.0,0.0,8.0,4.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42303,84610,243,10,1.0,3,14,0,N,N,2012-12-01,42,2012-04-23,2012,FT. WRIGHT,KY,OH,390000,150000,127500,0,1,3,850,17,35,3,3_42,XX,XX,100000.0,290000.0,88320.0,61680.0,58320.0,69180.0,0.0,3.0,0.0,14.0,6.0,4.0,96200.0,293800.0,80000.0,70000.0,56250.0,71250.0,0.0,3.0,0.0,14.0,4.0,6.0,98500.0,291500.0,83000.0,67000.0,57600.0,69900.0,0.0,3.0,0.0,14.0,4.0,6.0
42304,84611,178,0,2.0,0,0,1,N,N,2003-11-30,0,2006-10-27,2007,PHILADELPHIA,PA,RI,100000,100000,90000,0,1,3,1848,38,40,1,1_0,PA_0,PHILADELPHIA_0,80000.0,20000.0,80000.0,20000.

In [25]:
def create_svd(col,train_df,test_df,feature):
    svd=TruncatedSVD(n_components=len(col)//5,random_state=1)
    tmp_train_df=train_df[col]
    tmp_test_df=test_df[col]
#     df=pd.concat([tmp_train_df,tmp_test_df])
    df=tmp_train_df.copy()
    ss=StandardScaler()
    df[col]=ss.fit_transform(df[col])
    tmp_train_df[col]=ss.transform(tmp_train_df[col])
    tmp_test_df[col]=ss.transform(tmp_test_df[col])
    svd.fit(df)
    col_name='svd'
    for c in col:
        c=f'{c}'
        col_name+=c
    tmp_train_df=svd.transform(tmp_train_df)
    tmp_test_df=svd.transform(tmp_test_df)

    tmp_train_df=pd.DataFrame(tmp_train_df)
    tmp_test_df=pd.DataFrame(tmp_test_df)
    tmp_train_df = tmp_train_df.add_prefix(f'svd_{feature}_')
    tmp_test_df = tmp_test_df.add_prefix(f'svd_{feature}_')
    train_df=pd.concat([train_df,tmp_train_df],axis=1)
    test_df=pd.concat([test_df,tmp_test_df],axis=1)
    return train_df,test_df
def create_nmf(col,train_df,test_df,feature):
    nmf=NMF(n_components=len(col)//5,random_state=1)
#     nmf=NMF(n_components=s,random_state=1)
    tmp_train_df=train_df[col]
    tmp_test_df=test_df[col]
#     df=pd.concat([tmp_train_df,tmp_test_df])
    df=tmp_train_df.copy()
    mm=MinMaxScaler(feature_range=(0, 100),clip=True)
    ss=StandardScaler()
    df[col]=ss.fit_transform(df[col])
    tmp_train_df[col]=ss.transform(tmp_train_df[col])
    tmp_test_df[col]=ss.transform(tmp_test_df[col])
    df[col]=mm.fit_transform(df[col])
    tmp_train_df[col]=mm.transform(tmp_train_df[col])
    tmp_test_df[col]=mm.transform(tmp_test_df[col])
    nmf.fit(df)
    col_name='nmf'
    for c in col:
        c=f'{c}'
        col_name+=c
    tmp_train_df=nmf.transform(tmp_train_df)
    tmp_test_df=nmf.transform(tmp_test_df)
    
    tmp_train_df=pd.DataFrame(tmp_train_df)
    tmp_test_df=pd.DataFrame(tmp_test_df)
    tmp_train_df = tmp_train_df.add_prefix(f'nmf_{feature}_')
    tmp_test_df = tmp_test_df.add_prefix(f'nmf_{feature}_')
    train_df=pd.concat([train_df,tmp_train_df],axis=1)
    test_df=pd.concat([test_df,tmp_test_df],axis=1)
    return train_df,test_df

In [26]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='infrequent_if_exist',min_frequency=100,max_categories=100,sparse_output=False)
train_df_city=enc.fit_transform(train_df[['City']])
test_df_city=enc.transform(test_df[['City']])
train_df_city=pd.DataFrame(train_df_city).add_prefix('City_col')
test_df_city=pd.DataFrame(test_df_city).add_prefix('City_col')
col_list=train_df_city.columns
train_df=pd.concat([train_df,train_df_city],axis=1)
test_df=pd.concat([test_df,test_df_city],axis=1)

train_df,test_df = create_svd(col_list,train_df,test_df,'City')
train_df,test_df = create_nmf(col_list,train_df,test_df,'City')
train_df.drop(col_list,axis=1,inplace=True)
test_df.drop(col_list,axis=1,inplace=True)

In [27]:
del c,dataFrame,eda_df,test_df_city,train_df_city
gc.collect()

79

In [28]:
train_df_12

,Unnamed: 0,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,RevLineCr,LowDoc,DisbursementDate,MIS_Status,Sector,ApprovalDate,ApprovalFY,City,State,BankState,DisbursementGross,GrAppv,SBA_Appv,UrbanRural,RevLineCr_label,LowDoc_label,City_label,State_label,BankState_label,NoEmp_label,main_key_NoEmp_label,main_key_State,main_key_City,DisbursementGross_main_key_NoEmp_label_med,DisbursementGross_main_key_NoEmp_label_diff,GrAppv_main_key_NoEmp_label_med,GrAppv_main_key_NoEmp_label_diff,SBA_Appv_main_key_NoEmp_label_med,SBA_Appv_main_key_NoEmp_label_diff,CreateJob_main_key_NoEmp_label_med,CreateJob_main_key_NoEmp_label_diff,RetainedJob_main_key_NoEmp_label_med,RetainedJob_main_key_NoEmp_label_diff,NoEmp_main_key_NoEmp_label_med,NoEmp_main_key_NoEmp_label_diff,DisbursementGross_main_key_State_med,DisbursementGross_main_key_State_diff,GrAppv_main_key_State_med,GrAppv_main_key_State_diff,SBA_Appv_main_key_State_med,SBA_Appv_main_key_State_diff,CreateJob_main_key_State_med,CreateJob_main_key_State_diff,RetainedJob_main_key_State_med,RetainedJob_main_key_State_diff,NoEmp_main_key_State_med,NoEmp_main_key_State_diff,DisbursementGross_main_key_City_med,DisbursementGross_main_key_City_diff,GrAppv_main_key_City_med,GrAppv_main_key_City_diff,SBA_Appv_main_key_City_med,SBA_Appv_main_key_City_diff,CreateJob_main_key_City_med,CreateJob_main_key_City_diff,RetainedJob_main_key_City_med,RetainedJob_main_key_City_diff,NoEmp_main_key_City_med,NoEmp_main_key_City_diff
0,2,242,45,1.0,4,90,0,N,N,2001-08-31,1,42,2001-04-18,2001,HAWTHORNE,NJ,NJ,31983,30000,15000,1,1,3,634,31,31,4,4_42,NJ_42,XX,121000.0,-89017.0,100000.0,-70000.0,61000.0,-46000.0,1.0,3.0,1.0,89.0,18.0,27.0,76596.0,-44613.0,62340.0,-32340.0,36750.0,-21750.0,0.0,4.0,3.0,87.0,10.0,35.0,94203.0,-62220.0,80000.0,-50000.0,53864.5,-38864.5,0.0,4.0,1.0,89.0,3.0,42.0
1,9,25,1,1.0,0,0,1,N,N,2007-04-30,1,0,2004-03-23,2004,MESA,AZ,AZ,50000,50000,25000,2,1,3,928,3,3,1,1_0,XX,XX,80000.0,-30000.0,80000.0,-30000.0,61500.0,-36500.0,0.0,0.0,0.0,0.0,1.0,0.0,86787.0,-36787.0,75000.0,-25000.0,51000.0,-26000.0,0.0,0.0,1.0,-1.0,3.0,-2.0,94203.0,-44203.0,80000.0,-30000.0,53864.5,-28864.5,0.0,0.0,1.0,-1.0,3.0,-2.0
2,11,23,0,2.0,0,1,0,Y,N,2003-08-31,1,62,2001-10-10,2002,GREENSBURG,PA,RI,400000,600000,450000,1,3,3,604,38,39,1,1_62,XX,XX,100000.0,300000.0,100000.0,500000.0,71250.0,378750.0,0.0,0.0,0.0,1.0,2.0,-2.0,86787.0,313213.0,75000.0,525000.0,51000.0,399000.0,0.0,0.0,1.0,0.0,3.0,-3.0,94203.0,305797.0,80000.0,520000.0,53864.5,396135.5,0.0,0.0,1.0,0.0,3.0,-3.0
3,12,56,0,2.0,1,1,57300,Y,NaN,2006-01-31,0,61,2003-06-06,2003,NORTH SALT LAKE,UT,UT,100000,100000,50000,2,3,6,1075,44,44,1,1_61,XX,XX,80000.0,20000.0,75000.0,25000.0,60000.0,-10000.0,0.0,1.0,0.0,1.0,1.0,-1.0,86787.0,13213.0,75000.0,25000.0,51000.0,-1000.0,0.0,1.0,1.0,0.0,3.0,-3.0,94203.0,5797.0,80000.0,20000.0,53864.5,-3864.5,0.0,1.0,1.0,0.0,3.0,-3.0
4,13,237,2,1.0,0,12,0,N,N,1999-08-31,1,72,1996-09-06,1996,FOND DU LAC,WI,WI,323000,323000,323000,1,1,3,510,48,48,1,1_72,XX,XX,66000.0,257000.0,60000.0,263000.0,47050.0,275950.0,0.0,0.0,1.0,11.0,1.0,1.0,86787.0,236213.0,75000.0,248000.0,51000.0,272000.0,0.0,0.0,1.0,11.0,3.0,-1.0,94203.0,228797.0,80000.0,243000.0,53864.5,269135.5,0.0,0.0,1.0,11.0,3.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18265,42295,11,17,1.0,0,0,0,N,N,2008-05-31,1,42,2011-04-19,2011,ROOSEVELT,UT,UT,30500,30500,15250,1,1,3,1290,44,44,4,4_42,UT_42,XX,121000.0,-90500.0,100000.0,-69500.0,61000.0,-45750.0,1.0,-1.0,1.0,-1.0,18.0,-1.0,63927.0,-33427.0,56000.0,-25500.0,42150.0,-26900.0,0.0,0.0,1.0,-1.0,4.0,13.0,94203.0,-63703.0,80000.0,-49500.0,53864.5,-38614.5,0.0,0.0,1.0,-1.0,3.0,14.0
18266,42296,234,6,1.0,0,1,0,Y,N,1999-10-31,1,0,2007-05-10,2007,ESCONDIDO,CA,CA,28500,28500,14250,1,3,3,466,4,4,3,3_0,CA_0,ESCONDIDO_0

# Create Model

In [29]:
train_df.columns

Index(['Unnamed: 0', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'RevLineCr', 'LowDoc', 'DisbursementDate',
       'MIS_Status', 'Sector', 'ApprovalDate', 'ApprovalFY', 'City', 'State',
       'BankState', 'DisbursementGross', 'GrAppv', 'SBA_Appv', 'UrbanRural',
       'RevLineCr_label', 'LowDoc_label', 'City_label', 'State_label',
       'BankState_label', 'NoEmp_label', 'main_key_NoEmp_label',
       'main_key_State', 'main_key_City',
       'DisbursementGross_main_key_NoEmp_label_med',
       'DisbursementGross_main_key_NoEmp_label_diff',
       'GrAppv_main_key_NoEmp_label_med', 'GrAppv_main_key_NoEmp_label_diff',
       'SBA_Appv_main_key_NoEmp_label_med',
       'SBA_Appv_main_key_NoEmp_label_diff',
       'CreateJob_main_key_NoEmp_label_med',
       'CreateJob_main_key_NoEmp_label_diff',
       'RetainedJob_main_key_NoEmp_label_med',
       'RetainedJob_main_key_NoEmp_label_diff',
       'NoEmp_main_key_NoEmp_label_med', 'NoEmp_main_key_NoEmp

In [30]:
drop_list = list(train_df.select_dtypes("object").columns)
drop_list.extend(train_df.select_dtypes("datetime").columns)
drop_list

['RevLineCr',
 'LowDoc',
 'City',
 'State',
 'BankState',
 'main_key_NoEmp_label',
 'main_key_State',
 'main_key_City',
 'DisbursementDate',
 'ApprovalDate']

In [31]:
target_encode_list = ["State","BankState"]
for c in target_encode_list:
    drop_list.remove(c)

In [32]:
drop_list

['RevLineCr',
 'LowDoc',
 'City',
 'main_key_NoEmp_label',
 'main_key_State',
 'main_key_City',
 'DisbursementDate',
 'ApprovalDate']

In [33]:
drop_list_other = ["Unnamed: 0","State_label","BankState_label","MIS_Status"]
[drop_list.append(c) for c in drop_list_other]

[None, None, None, None]

In [34]:
drop_list

['RevLineCr',
 'LowDoc',
 'City',
 'main_key_NoEmp_label',
 'main_key_State',
 'main_key_City',
 'DisbursementDate',
 'ApprovalDate',
 'Unnamed: 0',
 'State_label',
 'BankState_label',
 'MIS_Status']

In [35]:
from sklearn.metrics import log_loss
##
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)
##
def objective(train_x,train_y,test_x,test_y,valid_index,train_df,trial):
        d_train=lgb.Dataset(train_x,label=train_y)
        d_valid=lgb.Dataset(test_x,label=test_y)
        param = {
        'objective': 'binary',#trial.suggest_categorical("objective",['binary', 'tweedie']),'num_class': 3
        'metric': 'binary_logloss',
        'learning_rate': 0.003,#trial.suggest_loguniform('learning_rate',0.006),
        'boosting':'gbdt',
#         'is_unbalance':True,
#         'scale_pos_weight':trial.suggest_loguniform('scale_pos_weight', 1,5),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
#         'max_depth':trial.suggest_int('max_depth', 1, 10),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 100),
        'early_stopping_rounds':500,
        'seed':1,'verbose' : -1,'num_boost_round':100000,
        'tree_learner':trial.suggest_categorical("tree_learner",['serial', 'feature', 'data', 'voting']),
        # 'data_sample_strategy':trial.suggest_categorical("data_sample_strategy",['goss', 'bagging'])
    }
 
        gbm = lgb.train(param,d_train,valid_sets = [d_train, d_valid],callbacks=[lgb.early_stopping(stopping_rounds=500, verbose=True), lgb.log_evaluation(300)])
        preds = gbm.predict(test_x)
        
#         oof_predictions = np.zeros([len(train_df),2])
#         oof_predictions[valid_index]=preds
#         y_true=np.zeros([len(test_y),3])
#         y_true[:, 0]=np.where(test_y==0,1,0)
#         y_true[:, 1]=np.where(test_y==1,1,0)
#         y_true[:, 2]=np.where(test_y==2,1,0)
#         print(gbm.predict(test_x))
#         print(preds[:,0])
#         print(y_true[:,0])
#         print(preds.shape)
        score=log_loss(test_y,preds)
#         score=log_loss(test_y,preds)
        return score

In [36]:
target_encode_list = ["State","BankState"]

In [37]:
features=[c for c in train_df.columns if c not in drop_list]
# if CFG.model=='lgb':
import lightgbm as lgb
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
importance_df=pd.DataFrame(index=train_df[features].columns)
importance_df['importance']=0
fold=5
predictions=pd.DataFrame()
predict=np.zeros([len(test_df),])
oof_predictions = np.zeros([len(train_df),])
# Target Encoding for state and bank state test data  
for c in target_encode_list:
    tmp_df = pd.DataFrame({c:train_df[c],f"target_{c}":train_df["MIS_Status"]}) 
    target_mean = tmp_df.groupby(c)[f"target_{c}"].mean()
    test_df.loc[:,c] = test_df[c].map(target_mean)
    test_df[c] = test_df[c].astype(float)
    
for seeds in[1]:
    fold=5
    skf=MultilabelStratifiedKFold(n_splits=5,random_state=seeds,shuffle=True)
    selected=features.copy()
        
    for i ,(tr_index,va_index) in enumerate(skf.split(train_df,train_df[['MIS_Status','Sector','State','UrbanRural']])):


        tr_x,tr_y=train_df[selected].iloc[tr_index],train_df['MIS_Status'].iloc[tr_index]
        va_x,va_y=train_df[selected].iloc[va_index],train_df['MIS_Status'].iloc[va_index]

        # Target Encoding for state and bank state 
        for c in target_encode_list:
            tmp_df = pd.DataFrame({c:tr_x[c],f"target_{c}":tr_y}) 
            target_mean = tmp_df.groupby(c)[f"target_{c}"].mean()
            va_x.loc[:,c] = va_x[c].map(target_mean)

            tmp = np.repeat(np.nan, tr_x.shape[0])
            kf_encoding = KFold(n_splits=4, shuffle =True,random_state=seeds)

            for idx_1,idx_2 in kf_encoding.split(tr_x):
                target_mean = tmp_df.iloc[idx_1].groupby(c)[f"target_{c}"].mean()
                tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)
            tr_x.loc[:,c]=tmp

            tr_x[c] = tr_x[c].astype(float)
            va_x[c] = va_x[c].astype(float)
        d_train_df=lgb.Dataset(tr_x,label=tr_y)

        d_valid=lgb.Dataset(va_x,label=va_y)
        if CFG.train==False:
            param = {'objective': 'binary','metric': 'binary_logloss','learning_rate': 0.003,'boosting':'gbdt','scale_pos_weight': 1, 'lambda_l1': 1.8657458525452022, 'lambda_l2': 0.0043319561423214945, 'num_leaves': 20, 'feature_fraction': 0.23875565259514844, 'bagging_fraction': 0.7939326324321189, 'bagging_freq': 2, 'min_child_samples': 73, 'tree_learner': 'feature','early_stopping_rounds':500,
             'seed':seeds,'random_state':seeds,'data_random_seed':seeds,'feature_fraction_seed':seeds,'random_seed':seeds,'verbose' : -1,'num_boost_round':100000}

            gbm = lgb.train(param, d_train_df,valid_sets = [d_train_df, d_valid],callbacks=[lgb.early_stopping(stopping_rounds=500, verbose=True), lgb.log_evaluation(100)])
            preds=gbm.predict(va_x[selected])
            predict += gbm.predict(test_df[selected])/5
            oof_predictions[va_index]=preds
            importance_df['importance']+=gbm.feature_importance(importance_type='gain')/5
        else:
            if i==0:
                print(CFG.train)
                optuna.logging.set_verbosity(optuna.logging.INFO)
                study = optuna.create_study(direction='minimize')
                study.optimize(functools.partial(objective,tr_x,tr_y,va_x,va_y,va_index,train_df),timeout=60*60*3)#n_trials = 1000)
                print('Number of finished trials:', len(study.trials))
                print('Best trial:', study.best_trial.params)

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.319425	valid_1's binary_logloss: 0.321255
[200]	training's binary_logloss: 0.306678	valid_1's binary_logloss: 0.309492
[300]	training's binary_logloss: 0.297974	valid_1's binary_logloss: 0.30159
[400]	training's binary_logloss: 0.291843	valid_1's binary_logloss: 0.296283
[500]	training's binary_logloss: 0.287193	valid_1's binary_logloss: 0.292422
[600]	training's binary_logloss: 0.2837	valid_1's binary_logloss: 0.289664
[700]	training's binary_logloss: 0.280799	valid_1's binary_logloss: 0.287479
[800]	training's binary_logloss: 0.278301	valid_1's binary_logloss: 0.285677
[900]	training's binary_logloss: 0.276123	valid_1's binary_logloss: 0.284214
[1000]	training's binary_logloss: 0.274334	valid_1's binary_logloss: 0.283112
[1100]	training's binary_logloss: 0.272676	valid_1's binary_logloss: 0.282225
[1200]	training's binary_logloss: 0.271187	valid_1's binary_logloss: 0.281505
[1300]	trainin

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.31981	valid_1's binary_logloss: 0.319302
[200]	training's binary_logloss: 0.307203	valid_1's binary_logloss: 0.306857
[300]	training's binary_logloss: 0.298617	valid_1's binary_logloss: 0.298661
[400]	training's binary_logloss: 0.292511	valid_1's binary_logloss: 0.292989
[500]	training's binary_logloss: 0.287941	valid_1's binary_logloss: 0.2889
[600]	training's binary_logloss: 0.284467	valid_1's binary_logloss: 0.286031
[700]	training's binary_logloss: 0.281616	valid_1's binary_logloss: 0.283834
[800]	training's binary_logloss: 0.279116	valid_1's binary_logloss: 0.281917
[900]	training's binary_logloss: 0.276893	valid_1's binary_logloss: 0.280332
[1000]	training's binary_logloss: 0.275087	valid_1's binary_logloss: 0.27926
[1100]	training's binary_logloss: 0.273434	valid_1's binary_logloss: 0.278348
[1200]	training's binary_logloss: 0.271935	valid_1's binary_logloss: 0.277574
[1300]	training

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.318915	valid_1's binary_logloss: 0.32134
[200]	training's binary_logloss: 0.305884	valid_1's binary_logloss: 0.310275
[300]	training's binary_logloss: 0.296902	valid_1's binary_logloss: 0.303017
[400]	training's binary_logloss: 0.290544	valid_1's binary_logloss: 0.298166
[500]	training's binary_logloss: 0.285778	valid_1's binary_logloss: 0.294732
[600]	training's binary_logloss: 0.28221	valid_1's binary_logloss: 0.292416
[700]	training's binary_logloss: 0.279296	valid_1's binary_logloss: 0.290725
[800]	training's binary_logloss: 0.276721	valid_1's binary_logloss: 0.289339
[900]	training's binary_logloss: 0.274484	valid_1's binary_logloss: 0.288274
[1000]	training's binary_logloss: 0.272628	valid_1's binary_logloss: 0.287541
[1100]	training's binary_logloss: 0.27093	valid_1's binary_logloss: 0.286927
[1200]	training's binary_logloss: 0.269417	valid_1's binary_logloss: 0.28645
[1300]	training

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.319315	valid_1's binary_logloss: 0.32091
[200]	training's binary_logloss: 0.306436	valid_1's binary_logloss: 0.3092
[300]	training's binary_logloss: 0.297654	valid_1's binary_logloss: 0.301463
[400]	training's binary_logloss: 0.291488	valid_1's binary_logloss: 0.296141
[500]	training's binary_logloss: 0.286863	valid_1's binary_logloss: 0.292385
[600]	training's binary_logloss: 0.28338	valid_1's binary_logloss: 0.289727
[700]	training's binary_logloss: 0.280483	valid_1's binary_logloss: 0.287608
[800]	training's binary_logloss: 0.277949	valid_1's binary_logloss: 0.285936
[900]	training's binary_logloss: 0.275753	valid_1's binary_logloss: 0.284594
[1000]	training's binary_logloss: 0.273951	valid_1's binary_logloss: 0.283647
[1100]	training's binary_logloss: 0.272308	valid_1's binary_logloss: 0.282787
[1200]	training's binary_logloss: 0.270822	valid_1's binary_logloss: 0.282138
[1300]	training

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.31957	valid_1's binary_logloss: 0.320303
[200]	training's binary_logloss: 0.3069	valid_1's binary_logloss: 0.308056
[300]	training's binary_logloss: 0.298255	valid_1's binary_logloss: 0.299834
[400]	training's binary_logloss: 0.292119	valid_1's binary_logloss: 0.29413
[500]	training's binary_logloss: 0.28752	valid_1's binary_logloss: 0.290144
[600]	training's binary_logloss: 0.28405	valid_1's binary_logloss: 0.287279
[700]	training's binary_logloss: 0.281206	valid_1's binary_logloss: 0.285045
[800]	training's binary_logloss: 0.278665	valid_1's binary_logloss: 0.283192
[900]	training's binary_logloss: 0.276456	valid_1's binary_logloss: 0.281682
[1000]	training's binary_logloss: 0.274649	valid_1's binary_logloss: 0.280599
[1100]	training's binary_logloss: 0.272987	valid_1's binary_logloss: 0.279708
[1200]	training's binary_logloss: 0.27148	valid_1's binary_logloss: 0.27896
[1300]	training's b

In [38]:
importance_df.sort_values('importance',ascending=False).to_csv(f'importance_baseline_{CFG.ver}.csv')

In [39]:
importance_df

,importance
Term,34240.242577
NoEmp,28551.699413
NewExist,11412.940266
CreateJob,19658.823704
RetainedJob,14831.076687
FranchiseCode,9981.699911
Sector,7973.241156
ApprovalFY,12754.764381
State,17657.678100
BankState,23477.822279


In [40]:
features=[c for c in train_df_0.columns if c not in drop_list]
# if CFG.model=='lgb':
import lightgbm as lgb
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
importance_df_0=pd.DataFrame(index=train_df_0[features].columns)
importance_df_0['importance']=0
fold=5
predictions_0=pd.DataFrame()
predict_0=np.zeros([len(test_df_0),])
oof_predictions_0 = np.zeros([len(train_df_0),])
# Target Encoding for state and bank state test data  
for c in target_encode_list:
    tmp_df = pd.DataFrame({c:train_df_0[c],f"target_{c}":train_df_0["MIS_Status"]}) 
    target_mean = tmp_df.groupby(c)[f"target_{c}"].mean()
    test_df_0.loc[:,c] = test_df_0[c].map(target_mean)
    test_df_0[c] = test_df_0[c].astype(float)
    
for seeds in[1]:
    fold=5
    skf=MultilabelStratifiedKFold(n_splits=5,random_state=seeds,shuffle=True)
    selected=features.copy()
        
    for i ,(tr_index,va_index) in enumerate(skf.split(train_df_0,train_df_0[['MIS_Status','Sector','State','UrbanRural']])):


        tr_x,tr_y=train_df_0[selected].iloc[tr_index],train_df_0['MIS_Status'].iloc[tr_index]
        va_x,va_y=train_df_0[selected].iloc[va_index],train_df_0['MIS_Status'].iloc[va_index]

        # Target Encoding for state and bank state 
        for c in target_encode_list:
            tmp_df = pd.DataFrame({c:tr_x[c],f"target_{c}":tr_y}) 
            target_mean = tmp_df.groupby(c)[f"target_{c}"].mean()
            va_x.loc[:,c] = va_x[c].map(target_mean)

            tmp = np.repeat(np.nan, tr_x.shape[0])
            kf_encoding = KFold(n_splits=4, shuffle =True,random_state=seeds)

            for idx_1,idx_2 in kf_encoding.split(tr_x):
                target_mean = tmp_df.iloc[idx_1].groupby(c)[f"target_{c}"].mean()
                tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)
            tr_x.loc[:,c]=tmp

            tr_x[c] = tr_x[c].astype(float)
            va_x[c] = va_x[c].astype(float)
        d_train_df_0=lgb.Dataset(tr_x,label=tr_y)

        d_valid=lgb.Dataset(va_x,label=va_y)
        if CFG.train==False:
            param = {'objective': 'binary','metric': 'binary_logloss','learning_rate': 0.003,'boosting':'gbdt','scale_pos_weight': 1, 'lambda_l1': 1.8657458525452022, 'lambda_l2': 0.0043319561423214945, 'num_leaves': 20, 'feature_fraction': 0.23875565259514844, 'bagging_fraction': 0.7939326324321189, 'bagging_freq': 2, 'min_child_samples': 73, 'tree_learner': 'feature','early_stopping_rounds':500,
             'seed':seeds,'random_state':seeds,'data_random_seed':seeds,'feature_fraction_seed':seeds,'random_seed':seeds,'verbose' : -1,'num_boost_round':100000}

            gbm = lgb.train(param, d_train_df_0,valid_sets = [d_train_df_0, d_valid],callbacks=[lgb.early_stopping(stopping_rounds=500, verbose=True), lgb.log_evaluation(100)])
            preds=gbm.predict(va_x[selected])
            predict_0 += gbm.predict(test_df_0[selected])/5
            oof_predictions_0[va_index]=preds
            importance_df_0['importance']+=gbm.feature_importance(importance_type='gain')/5
        else:
            if i==0:
                print(CFG.train)
                optuna.logging.set_verbosity(optuna.logging.INFO)
                study = optuna.create_study(direction='minimize')
                study.optimize(functools.partial(objective,tr_x,tr_y,va_x,va_y,va_index,train_df_0),timeout=60*60*3)#n_trials = 1000)
                print('Number of finished trials:', len(study.trials))
                print('Best trial:', study.best_trial.params)

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.20659	valid_1's binary_logloss: 0.209049
[200]	training's binary_logloss: 0.202688	valid_1's binary_logloss: 0.207212
[300]	training's binary_logloss: 0.19957	valid_1's binary_logloss: 0.205823
[400]	training's binary_logloss: 0.197078	valid_1's binary_logloss: 0.20502
[500]	training's binary_logloss: 0.194873	valid_1's binary_logloss: 0.204384
[600]	training's binary_logloss: 0.192902	valid_1's binary_logloss: 0.203848
[700]	training's binary_logloss: 0.191071	valid_1's binary_logloss: 0.20345
[800]	training's binary_logloss: 0.189405	valid_1's binary_logloss: 0.203095
[900]	training's binary_logloss: 0.18784	valid_1's binary_logloss: 0.202852
[1000]	training's binary_logloss: 0.186341	valid_1's binary_logloss: 0.202658
[1100]	training's binary_logloss: 0.184934	valid_1's binary_logloss: 0.202498
[1200]	training's binary_logloss: 0.183585	valid_1's binary_logloss: 0.202438
[1300]	training'

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.206902	valid_1's binary_logloss: 0.208189
[200]	training's binary_logloss: 0.203072	valid_1's binary_logloss: 0.205606
[300]	training's binary_logloss: 0.200061	valid_1's binary_logloss: 0.203715
[400]	training's binary_logloss: 0.197644	valid_1's binary_logloss: 0.202324
[500]	training's binary_logloss: 0.195516	valid_1's binary_logloss: 0.201323
[600]	training's binary_logloss: 0.193606	valid_1's binary_logloss: 0.200566
[700]	training's binary_logloss: 0.191773	valid_1's binary_logloss: 0.1999
[800]	training's binary_logloss: 0.19008	valid_1's binary_logloss: 0.199416
[900]	training's binary_logloss: 0.188521	valid_1's binary_logloss: 0.199013
[1000]	training's binary_logloss: 0.18706	valid_1's binary_logloss: 0.198756
[1100]	training's binary_logloss: 0.185673	valid_1's binary_logloss: 0.198406
[1200]	training's binary_logloss: 0.184335	valid_1's binary_logloss: 0.198181
[1300]	training

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.20675	valid_1's binary_logloss: 0.208907
[200]	training's binary_logloss: 0.202792	valid_1's binary_logloss: 0.206517
[300]	training's binary_logloss: 0.1997	valid_1's binary_logloss: 0.204792
[400]	training's binary_logloss: 0.197224	valid_1's binary_logloss: 0.203726
[500]	training's binary_logloss: 0.195092	valid_1's binary_logloss: 0.203042
[600]	training's binary_logloss: 0.193124	valid_1's binary_logloss: 0.20251
[700]	training's binary_logloss: 0.191319	valid_1's binary_logloss: 0.202164
[800]	training's binary_logloss: 0.189619	valid_1's binary_logloss: 0.20183
[900]	training's binary_logloss: 0.188045	valid_1's binary_logloss: 0.201536
[1000]	training's binary_logloss: 0.186579	valid_1's binary_logloss: 0.201334
[1100]	training's binary_logloss: 0.185114	valid_1's binary_logloss: 0.201217
[1200]	training's binary_logloss: 0.183759	valid_1's binary_logloss: 0.201075
[1300]	training'

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.206542	valid_1's binary_logloss: 0.2096
[200]	training's binary_logloss: 0.202411	valid_1's binary_logloss: 0.207571
[300]	training's binary_logloss: 0.199141	valid_1's binary_logloss: 0.206259
[400]	training's binary_logloss: 0.196525	valid_1's binary_logloss: 0.20545
[500]	training's binary_logloss: 0.194247	valid_1's binary_logloss: 0.204846
[600]	training's binary_logloss: 0.192208	valid_1's binary_logloss: 0.204472
[700]	training's binary_logloss: 0.190367	valid_1's binary_logloss: 0.204127
[800]	training's binary_logloss: 0.188617	valid_1's binary_logloss: 0.203916
[900]	training's binary_logloss: 0.187044	valid_1's binary_logloss: 0.203677
[1000]	training's binary_logloss: 0.185513	valid_1's binary_logloss: 0.203558
[1100]	training's binary_logloss: 0.184052	valid_1's binary_logloss: 0.203522
[1200]	training's binary_logloss: 0.182717	valid_1's binary_logloss: 0.2035
[1300]	training'

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.206724	valid_1's binary_logloss: 0.208162
[200]	training's binary_logloss: 0.202599	valid_1's binary_logloss: 0.205905
[300]	training's binary_logloss: 0.199311	valid_1's binary_logloss: 0.204345
[400]	training's binary_logloss: 0.196842	valid_1's binary_logloss: 0.203453
[500]	training's binary_logloss: 0.19466	valid_1's binary_logloss: 0.202767
[600]	training's binary_logloss: 0.192711	valid_1's binary_logloss: 0.202197
[700]	training's binary_logloss: 0.190983	valid_1's binary_logloss: 0.201777
[800]	training's binary_logloss: 0.189313	valid_1's binary_logloss: 0.201435
[900]	training's binary_logloss: 0.187751	valid_1's binary_logloss: 0.201144
[1000]	training's binary_logloss: 0.18629	valid_1's binary_logloss: 0.200888
[1100]	training's binary_logloss: 0.18489	valid_1's binary_logloss: 0.20067
[1200]	training's binary_logloss: 0.183562	valid_1's binary_logloss: 0.200514
[1300]	training

In [41]:
features=[c for c in train_df_12.columns if c not in drop_list]
# if CFG.model=='lgb':
import lightgbm as lgb
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
importance_df_12=pd.DataFrame(index=train_df_12[features].columns)
importance_df_12['importance']=0
fold=5
predictions_12=pd.DataFrame()
predict_12=np.zeros([len(test_df_12),])
oof_predictions_12 = np.zeros([len(train_df_12),])
# Target Encoding for state and bank state test data  
for c in target_encode_list:
    tmp_df = pd.DataFrame({c:train_df_12[c],f"target_{c}":train_df_12["MIS_Status"]}) 
    target_mean = tmp_df.groupby(c)[f"target_{c}"].mean()
    test_df_12.loc[:,c] = test_df_12[c].map(target_mean)
    test_df_12[c] = test_df_12[c].astype(float)
    
for seeds in[1]:
    fold=5
    skf=MultilabelStratifiedKFold(n_splits=5,random_state=seeds,shuffle=True)
    selected=features.copy()
        
    for i ,(tr_index,va_index) in enumerate(skf.split(train_df_12,train_df_12[['MIS_Status','Sector','State','UrbanRural']])):


        tr_x,tr_y=train_df_12[selected].iloc[tr_index],train_df_12['MIS_Status'].iloc[tr_index]
        va_x,va_y=train_df_12[selected].iloc[va_index],train_df_12['MIS_Status'].iloc[va_index]

        # Target Encoding for state and bank state 
        for c in target_encode_list:
            tmp_df = pd.DataFrame({c:tr_x[c],f"target_{c}":tr_y}) 
            target_mean = tmp_df.groupby(c)[f"target_{c}"].mean()
            va_x.loc[:,c] = va_x[c].map(target_mean)

            tmp = np.repeat(np.nan, tr_x.shape[0])
            kf_encoding = KFold(n_splits=4, shuffle =True,random_state=seeds)

            for idx_1,idx_2 in kf_encoding.split(tr_x):
                target_mean = tmp_df.iloc[idx_1].groupby(c)[f"target_{c}"].mean()
                tmp[idx_2] = tr_x[c].iloc[idx_2].map(target_mean)
            tr_x.loc[:,c]=tmp

            tr_x[c] = tr_x[c].astype(float)
            va_x[c] = va_x[c].astype(float)
        d_train_df_12=lgb.Dataset(tr_x,label=tr_y)

        d_valid=lgb.Dataset(va_x,label=va_y)
        if CFG.train==False:
            param = {'objective': 'binary','metric': 'binary_logloss','learning_rate': 0.003,'boosting':'gbdt','scale_pos_weight': 1, 'lambda_l1': 1.8657458525452022, 'lambda_l2': 0.0043319561423214945, 'num_leaves': 20, 'feature_fraction': 0.23875565259514844, 'bagging_fraction': 0.7939326324321189, 'bagging_freq': 2, 'min_child_samples': 73, 'tree_learner': 'feature','early_stopping_rounds':500,
             'seed':seeds,'random_state':seeds,'data_random_seed':seeds,'feature_fraction_seed':seeds,'random_seed':seeds,'verbose' : -1,'num_boost_round':100000}

            gbm = lgb.train(param, d_train_df_12,valid_sets = [d_train_df_12, d_valid],callbacks=[lgb.early_stopping(stopping_rounds=500, verbose=True), lgb.log_evaluation(100)])
            preds=gbm.predict(va_x[selected])
            predict_12 += gbm.predict(test_df_12[selected])/5
            oof_predictions_12[va_index]=preds
            importance_df_12['importance']+=gbm.feature_importance(importance_type='gain')/5
        else:
            if i==0:
                print(CFG.train)
                optuna.logging.set_verbosity(optuna.logging.INFO)
                study = optuna.create_study(direction='minimize')
                study.optimize(functools.partial(objective,tr_x,tr_y,va_x,va_y,va_index,train_df_12),timeout=60*60*3)#n_trials = 1000)
                print('Number of finished trials:', len(study.trials))
                print('Best trial:', study.best_trial.params)

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.440779	valid_1's binary_logloss: 0.442925
[200]	training's binary_logloss: 0.425627	valid_1's binary_logloss: 0.429582
[300]	training's binary_logloss: 0.413729	valid_1's binary_logloss: 0.419403
[400]	training's binary_logloss: 0.404629	valid_1's binary_logloss: 0.411942
[500]	training's binary_logloss: 0.397724	valid_1's binary_logloss: 0.406612
[600]	training's binary_logloss: 0.39185	valid_1's binary_logloss: 0.402204
[700]	training's binary_logloss: 0.387155	valid_1's binary_logloss: 0.399089
[800]	training's binary_logloss: 0.383074	valid_1's binary_logloss: 0.396593
[900]	training's binary_logloss: 0.379507	valid_1's binary_logloss: 0.394562
[1000]	training's binary_logloss: 0.3763	valid_1's binary_logloss: 0.392826
[1100]	training's binary_logloss: 0.373396	valid_1's binary_logloss: 0.391565
[1200]	training's binary_logloss: 0.370835	valid_1's binary_logloss: 0.390535
[1300]	trainin

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.441798	valid_1's binary_logloss: 0.440818
[200]	training's binary_logloss: 0.426951	valid_1's binary_logloss: 0.426167
[300]	training's binary_logloss: 0.41509	valid_1's binary_logloss: 0.414648
[400]	training's binary_logloss: 0.406044	valid_1's binary_logloss: 0.406348
[500]	training's binary_logloss: 0.399113	valid_1's binary_logloss: 0.400284
[600]	training's binary_logloss: 0.393264	valid_1's binary_logloss: 0.395381
[700]	training's binary_logloss: 0.388598	valid_1's binary_logloss: 0.391963
[800]	training's binary_logloss: 0.384546	valid_1's binary_logloss: 0.389081
[900]	training's binary_logloss: 0.380959	valid_1's binary_logloss: 0.386828
[1000]	training's binary_logloss: 0.377711	valid_1's binary_logloss: 0.384886
[1100]	training's binary_logloss: 0.374806	valid_1's binary_logloss: 0.383242
[1200]	training's binary_logloss: 0.372216	valid_1's binary_logloss: 0.381956
[1300]	train

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.43976	valid_1's binary_logloss: 0.446124
[200]	training's binary_logloss: 0.423848	valid_1's binary_logloss: 0.434348
[300]	training's binary_logloss: 0.411442	valid_1's binary_logloss: 0.425439
[400]	training's binary_logloss: 0.402073	valid_1's binary_logloss: 0.419041
[500]	training's binary_logloss: 0.394798	valid_1's binary_logloss: 0.41431
[600]	training's binary_logloss: 0.3886	valid_1's binary_logloss: 0.410441
[700]	training's binary_logloss: 0.383786	valid_1's binary_logloss: 0.407791
[800]	training's binary_logloss: 0.379671	valid_1's binary_logloss: 0.405764
[900]	training's binary_logloss: 0.375942	valid_1's binary_logloss: 0.404102
[1000]	training's binary_logloss: 0.372646	valid_1's binary_logloss: 0.402716
[1100]	training's binary_logloss: 0.3697	valid_1's binary_logloss: 0.401594
[1200]	training's binary_logloss: 0.367023	valid_1's binary_logloss: 0.400829
[1300]	training's

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.441467	valid_1's binary_logloss: 0.441382
[200]	training's binary_logloss: 0.426494	valid_1's binary_logloss: 0.427234
[300]	training's binary_logloss: 0.414762	valid_1's binary_logloss: 0.41643
[400]	training's binary_logloss: 0.405726	valid_1's binary_logloss: 0.408651
[500]	training's binary_logloss: 0.398786	valid_1's binary_logloss: 0.403007
[600]	training's binary_logloss: 0.392825	valid_1's binary_logloss: 0.398419
[700]	training's binary_logloss: 0.388027	valid_1's binary_logloss: 0.394988
[800]	training's binary_logloss: 0.383868	valid_1's binary_logloss: 0.392305
[900]	training's binary_logloss: 0.380275	valid_1's binary_logloss: 0.390215
[1000]	training's binary_logloss: 0.377059	valid_1's binary_logloss: 0.388458
[1100]	training's binary_logloss: 0.374088	valid_1's binary_logloss: 0.38702
[1200]	training's binary_logloss: 0.371474	valid_1's binary_logloss: 0.38586
[1300]	trainin

C:\Users\kaminogo.kenji\AppData\Local\miniconda3\envs\venv_tabledata_001\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 500 rounds
[100]	training's binary_logloss: 0.441138	valid_1's binary_logloss: 0.443411
[200]	training's binary_logloss: 0.42566	valid_1's binary_logloss: 0.430144
[300]	training's binary_logloss: 0.413555	valid_1's binary_logloss: 0.419985
[400]	training's binary_logloss: 0.404366	valid_1's binary_logloss: 0.412698
[500]	training's binary_logloss: 0.397341	valid_1's binary_logloss: 0.407515
[600]	training's binary_logloss: 0.391398	valid_1's binary_logloss: 0.403364
[700]	training's binary_logloss: 0.386746	valid_1's binary_logloss: 0.400385
[800]	training's binary_logloss: 0.382599	valid_1's binary_logloss: 0.397942
[900]	training's binary_logloss: 0.37901	valid_1's binary_logloss: 0.396017
[1000]	training's binary_logloss: 0.375822	valid_1's binary_logloss: 0.394461
[1100]	training's binary_logloss: 0.372925	valid_1's binary_logloss: 0.393098
[1200]	training's binary_logloss: 0.370337	valid_1's binary_logloss: 0.392097
[1300]	traini

In [125]:
oof_predictions

array([0.98278731, 0.9596817 , 0.97077445, ..., 0.93702945, 0.97641222,
       0.94820589])

In [126]:
oof_predictions_0

array([0.98909965, 0.97847322, 0.95707324, ..., 0.93856673, 0.9749496 ,
       0.95106083])

In [127]:
oof_predictions_12

array([0.97642248, 0.8481481 , 0.65950654, ..., 0.52811883, 0.90054612,
       0.9126719 ])

In [128]:
oof=pd.DataFrame(oof_predictions)
oof.rename(columns={0:'prob1'},inplace=True)
oof['True']=train['MIS_Status']

In [129]:
oof_predictions_0

array([0.98909965, 0.97847322, 0.95707324, ..., 0.93856673, 0.9749496 ,
       0.95106083])

In [130]:
oof_0=pd.DataFrame(oof_predictions_0)
oof_0.rename(columns={0:'prob2'},inplace=True)
oof_0['True']=train_df_0['MIS_Status']

In [131]:
oof_12=pd.DataFrame(oof_predictions_12)
oof_12.rename(columns={0:'prob2'},inplace=True)
oof_12['True']=train_df_12['MIS_Status']

In [132]:
oof_0 = pd.concat([oof_0,train_df_0["Unnamed: 0"]],axis=1)

In [133]:
oof_0

,prob2,True,Unnamed: 0
0,0.989100,1,0
1,0.978473,1,1
2,0.957073,1,3
3,0.953670,1,4
4,0.962035,0,5
...,...,...,...
24032,0.948376,1,42300
24033,0.979637,1,42302
24034,0.938567,1,42304
24035,0.974950,1,42305


In [134]:
oof_12

,prob2,True
0,0.976422,1
1,0.848148,1
2,0.659507,1
3,0.334563,0
4,0.828680,1
...,...,...
18265,0.933136,1
18266,0.756594,1
18267,0.528119,0
18268,0.900546,1


In [135]:
oof_12 = pd.concat([oof_12,train_df_12["Unnamed: 0"]],axis=1)

In [136]:
oof_12

,prob2,True,Unnamed: 0
0,0.976422,1,2
1,0.848148,1,9
2,0.659507,1,11
3,0.334563,0,12
4,0.828680,1,13
...,...,...,...
18265,0.933136,1,42295
18266,0.756594,1,42296
18267,0.528119,0,42299
18268,0.900546,1,42301


In [137]:
oof_012 = pd.concat([oof_0,oof_12])


In [138]:
oof_0

,prob2,True,Unnamed: 0
0,0.989100,1,0
1,0.978473,1,1
2,0.957073,1,3
3,0.953670,1,4
4,0.962035,0,5
...,...,...,...
24032,0.948376,1,42300
24033,0.979637,1,42302
24034,0.938567,1,42304
24035,0.974950,1,42305


In [139]:
oof_12

,prob2,True,Unnamed: 0
0,0.976422,1,2
1,0.848148,1,9
2,0.659507,1,11
3,0.334563,0,12
4,0.828680,1,13
...,...,...,...
18265,0.933136,1,42295
18266,0.756594,1,42296
18267,0.528119,0,42299
18268,0.900546,1,42301


In [140]:
oof_012 = oof_012.sort_values("Unnamed: 0")

In [141]:
oof_012

,prob2,True,Unnamed: 0
0,0.989100,1,0
1,0.978473,1,1
0,0.976422,1,2
2,0.957073,1,3
3,0.953670,1,4
...,...,...,...
24033,0.979637,1,42302
18269,0.912672,1,42303
24034,0.938567,1,42304
24035,0.974950,1,42305


In [142]:
oof_012.reset_index(inplace=True,drop=True)

In [143]:
oof_012

,prob2,True,Unnamed: 0
0,0.989100,1,0
1,0.978473,1,1
2,0.976422,1,2
3,0.957073,1,3
4,0.953670,1,4
...,...,...,...
42302,0.979637,1,42302
42303,0.912672,1,42303
42304,0.938567,1,42304
42305,0.974950,1,42305


# Value Metrics

In [144]:
from sklearn.metrics import f1_score

score_list = [{"Thresh":"Score"}]
for thresh in[0.6,0.7,0.74,0.745,0.746,0.747,0.748,0.749,0.75,0.751,0.752,0.753,0.754,0.755,0.76,0.761,0.762,0.763,0.767,0.768,0.769,0.77,0.771,0.772,0.773,0.774,0.78,0.8,0.9]:
    oof_012["thresh"] = 0
    oof_012['thresh']=np.where(oof_012['prob1']>thresh,1,0)
    print("#######")
    print(thresh)
    print(f1_score(oof_012['thresh'],oof_012['True'],average='macro'))
    score =f1_score(oof_012['thresh'],oof_012['True'],average='macro')
    score_list.append({f"{thresh}":f"{score}"})
    print("#######")

KeyError: 'prob1'

In [145]:
from sklearn.metrics import f1_score

score_list = [{"Thresh":"Score"}]
for thresh in[0.6,0.7,0.74,0.745,0.746,0.747,0.748,0.749,0.75,0.751,0.752,0.753,0.754,0.755,0.76,0.761,0.762,0.763,0.767,0.768,0.769,0.77,0.771,0.772,0.773,0.774,0.78,0.8,0.9]:
    oof["thresh"] = 0
    oof['thresh']=np.where(oof['prob1']>thresh,1,0)
    print("#######")
    print(thresh)
    print(f1_score(oof['thresh'],oof['True'],average='macro'))
    score =f1_score(oof['thresh'],oof['True'],average='macro')
    score_list.append({f"{thresh}":f"{score}"})
    print("#######")

#######
0.6
0.6551935539028755
#######
#######
0.7
0.6757262545461813
#######
#######
0.74
0.6783457799679364
#######
#######
0.745
0.6783683078111551
#######
#######
0.746
0.6782148012348871
#######
#######
0.747
0.6781746214435039
#######
#######
0.748
0.6782339634584447
#######
#######
0.749
0.67813330563663
#######
#######
0.75
0.67851494246713
#######
#######
0.751
0.6785121418835771
#######
#######
0.752
0.6783978743650534
#######
#######
0.753
0.6782118000788863
#######
#######
0.754
0.6784838173086946
#######
#######
0.755
0.6784023024209216
#######
#######
0.76
0.6775285887773324
#######
#######
0.761
0.6772910897810405
#######
#######
0.762
0.6773512883855957
#######
#######
0.763
0.6773568930550865
#######
#######
0.767
0.6773411835773543
#######
#######
0.768
0.6773383762758138
#######
#######
0.769
0.6772934848114873
#######
#######
0.77
0.6771121884868047
#######
#######
0.771
0.6771441862907119
#######
#######
0.772
0.6772026054334609
#######
#######
0.773
0.677228492323

In [146]:
oof = pd.concat([oof_012["prob2"],oof],axis=1)

In [147]:
oof

,prob2,prob1,True,thresh
0,0.989100,0.982787,1,1
1,0.978473,0.959682,1,1
2,0.976422,0.970774,1,1
3,0.957073,0.962944,1,1
4,0.953670,0.941949,1,1
...,...,...,...,...
42302,0.979637,0.983902,1,1
42303,0.912672,0.932100,1,1
42304,0.938567,0.937029,1,1
42305,0.974950,0.976412,1,1


In [148]:
score_df = pd.DataFrame(columns = ["score","thresh","prob1","prob2"])

In [149]:
oof

,prob2,prob1,True,thresh
0,0.989100,0.982787,1,1
1,0.978473,0.959682,1,1
2,0.976422,0.970774,1,1
3,0.957073,0.962944,1,1
4,0.953670,0.941949,1,1
...,...,...,...,...
42302,0.979637,0.983902,1,1
42303,0.912672,0.932100,1,1
42304,0.938567,0.937029,1,1
42305,0.974950,0.976412,1,1


In [150]:
from sklearn.metrics import f1_score


score_list = ["Thresh","Score","prob1","prob2"]
for thresh in[0.6,0.7,0.74,0.745,0.746,0.747,0.748,0.749,0.75,0.751,0.752,0.753,0.754,0.755,0.76,0.761,0.762,0.763,0.767,0.768,0.769,0.77,0.771,0.772,0.773,0.774,0.78,0.8,0.9]:
    for i,(prob1,prob2) in enumerate([(0.5,0.5),(0.4,0.6),(0.3,0.7),(0.6,0.4),(0.7,0.3)]):
        oof["thresh"] = 0
        oof["prob3"] = oof["prob1"]*prob1 + oof["prob2"]*prob2
        oof['thresh']=np.where(oof['prob3']>thresh,1,0)
        print("#######")
        print(thresh)
        print(f1_score(oof['thresh'],oof['True'],average='macro'))
        score =f1_score(oof['thresh'],oof['True'],average='macro')
        score_df.loc[i] = [score,thresh,prob1,prob2]
        print("#######")

#######
0.6
0.6518820840216917
#######
#######
0.6
0.6513754889623037
#######
#######
0.6
0.6508071097654289
#######
#######
0.6
0.652892978480133
#######
#######
0.6
0.6542747012325576
#######
#######
0.7
0.6763260373086678
#######
#######
0.7
0.6752735604387412
#######
#######
0.7
0.6742559899718332
#######
#######
0.7
0.6754029309933527
#######
#######
0.7
0.6760542992364853
#######
#######
0.74
0.6794783071320799
#######
#######
0.74
0.6795935845571851
#######
#######
0.74
0.6789227605569251
#######
#######
0.74
0.6790658669797827
#######
#######
0.74
0.6788494649182101
#######
#######
0.745
0.6795891596193958
#######
#######
0.745
0.679966019321282
#######
#######
0.745
0.6791326601544398
#######
#######
0.745
0.6797425973314202
#######
#######
0.745
0.6787096441026015
#######
#######
0.746
0.6796478962574708
#######
#######
0.746
0.6798772708891966
#######
#######
0.746
0.67932649082351
#######
#######
0.746
0.6797123021108289
#######
#######
0.746
0.6787293937876266
#######
####

In [151]:
score_df

,score,thresh,prob1,prob2
0,0.587150,0.9,0.5,0.5
1,0.586274,0.9,0.4,0.6
2,0.585699,0.9,0.3,0.7
3,0.587114,0.9,0.6,0.4
4,0.587545,0.9,0.7,0.3


# Create Submission

In [64]:
predict_0_df = pd.DataFrame(predict_0)
predict_12_df  = pd.DataFrame(predict_12) 

predict_0_df = pd.concat([predict_0_df,test_df_0["Unnamed: 0"]],axis=1)
predict_12_df  = pd.concat([predict_12_df,test_df_12["Unnamed: 0"]],axis=1)
predict_012_df = pd.concat([predict_0_df,predict_12_df])
predict_012_df.reset_index(inplace=True,drop=True)

In [71]:
# predict_012_df=predict_012_df.sort_values("Unnamed: 0")
# predict_012_df.reset_index(inplace=True,drop=True)
predict_012_df.rename(columns={0:"col_1"},inplace=True)

In [72]:
predict_012_df

,col_1,Unnamed: 0
0,0.294770,42307
1,0.960267,42308
2,0.606039,42309
3,0.821787,42310
4,0.945068,42311
...,...,...
42303,0.960238,84610
42304,0.941204,84611
42305,0.928225,84612
42306,0.965004,84613


In [73]:
pred_df=pd.DataFrame(predict)
pred_df=pred_df.add_prefix('col_')

In [74]:
pred_df

,col_0
0,0.310455
1,0.940694
2,0.721940
3,0.824712
4,0.953099
...,...
42303,0.964991
42304,0.947574
42305,0.920203
42306,0.962158


In [75]:
pred_df = pd.concat([pred_df,predict_012_df],axis=1)

In [76]:
pred_df[]

,col_0,col_1,Unnamed: 0
0,0.310455,0.294770,42307
1,0.940694,0.960267,42308
2,0.721940,0.606039,42309
3,0.824712,0.821787,42310
4,0.953099,0.945068,42311
...,...,...,...
42303,0.964991,0.960238,84610
42304,0.947574,0.941204,84611
42305,0.920203,0.928225,84612
42306,0.962158,0.965004,84613


In [99]:
pred_df=pd.DataFrame(predict)
pred_df=pred_df.add_prefix('col_')
sub=pd.DataFrame(test['Unnamed: 0'].copy())
# sub['predict_2']=0
sub['predict_0']=0
sub['predict_1']=0
sub['predict_1']=np.where(pred_df['col_0']>0.75,1,0)

# # sub['predict_2']=np.where(tmp['col_2']>0.0615,1,0)
# # sub['predict_0']=np.where(tmp['col_0']>0.205,1,0)

In [101]:
sub['predict_1'].value_counts()

predict_1
1    39043
0     3265
Name: count, dtype: int64

In [102]:
sub[['Unnamed: 0','predict_1']].to_csv(f'sub_kaminogo{CFG.ver}.csv',header=False,index=False)

# Other 

In [43]:
population_df= pd.read_csv("../data/us_popluation.csv")

In [23]:
population_df.dtypes

2024          int64
per_2024    float64
2023          int64
2010          int64
State        object
1900          int64
1910          int64
1920          int64
1930          int64
1940          int64
1950          int64
1960          int64
1970          int64
1980          int64
1990          int64
2000          int64
2003          int64
per_2023    float64
per_2010    float64
per_1900    float64
per_1910    float64
per_1920    float64
per_1930    float64
per_1940    float64
per_1950    float64
per_1960    float64
per_1970    float64
per_1980    float64
per_1990    float64
per_2000    float64
per_2003    float64
dtype: object

In [ ]:
population_df